<center style="font-size:2em;font-weight:bold">
   The Minimum Height of the Center of Mass
</center>
<p/>
<center style="font-size:1.75em;font-weight:bold">
   of a Right-Cylindrical Container of Liquid
</center>
<p/>
<center>Thomas E. Vaughan</center>

<!--- vim: set filetype=markdown tw=75: -->



# Introduction

In the shower one morning, I groggily reached for the shampoo, but I ended
up knocking the container onto its side.  I was surprised by my clumsiness
and woke up a bit.  I saw that there are two main reasons for the ease with
which I had knocked over the container.  One was obvious and uninteresting,
but the other was interesting enough to keep me thinking long after I
exited the shower.

The obvious, uninteresting reason is that the container was almost empty.
When a container has minimum content, its inertia is minimal.  Any
external, disturbing force has maximal effect on an object of minimal
inertia.

Yet, with water dripping off of my face and onto the floor, I stared down
at the container of shampoo as it lay sideways on the floor of the shower.
In that moment of thought, I realized that the low mass and low rotational
inertia of the nearly empty container did not satisfyingly explain what had
happened.  My disturbance of the container had been only very slight
indeed.  Even in its low-mass state, the container had not suffered much of
an angular perturbation before it fell over.  There was something more
interesting involved as well.

The container's stability against falling over depends on the height of the
contained liquid.  Before I finished with my shower, I had convinced myself
that both the completely empty state and the completely full state are the
least stable states.

<!--- vim: set filetype=markdown tw=75: -->



In [1]:
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
x = np.linspace(0, 2*np.pi)

def update(omega = 1.0):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(x, np.sin(omega*x))
    fig.canvas.draw()

interact(update)

interactive(children=(FloatSlider(value=1.0, description='omega', max=3.0, min=-1.0), Output()), _dom_classes=…

<function __main__.update(omega=1.0)>

In [3]:
import svgwrite as sw
from IPython import display
from IPython.display import SVG
import numpy as np

In [4]:
def drawing(wall_thickness = 0.1, # relative to wall height
            wall_density   = 1.0, # relative to fluid density
            base_width     = 0.5, # relative to wall height
            fluid_height   = 0.6  # relative to wall height
            ):
    sz = 200 # Size (pixels) of drawing.
    dr = sw.Drawing("drawing.svg", (sz, sz))
    
    # Draw left wall.
    dr.add(dr.rect((0, 0), (wall_thickness*sz, sz), fill='black'))
    
    # Draw right wall.
    dr.add(dr.rect(((base_width - wall_thickness)*sz, 0),
                   (wall_thickness*sz, sz), fill='black'))
    
    # Draw base.
    dr.add(dr.line((0, sz), (base_width*sz, sz), stroke='black'))
    
    # Draw fluid.
    surf = sz*(1.0 - fluid_height)
    dr.add(dr.rect((wall_thickness*sz, surf),
                   ((base_width - 2.0*wall_thickness)*sz, sz),
                   fill='blue'))
    
    # Draw line at height of center of mass.
    r1 = 0.5*base_width - wall_thickness
    r2 = 0.5*base_width
    wall_height   = 1
    fluid_density = 1
    wall_mass  =  wall_density*np.pi*(r2*r2 - r1*r1)* wall_height
    fluid_mass = fluid_density*np.pi*(r1*r1        )*fluid_height
    com = sz*(1.0 -
              (0.5*wall_height*wall_mass + 0.5*fluid_height*fluid_mass)/
              (wall_mass + fluid_mass))
    dr.add(dr.line((0, com), (base_width*sz, com), stroke='green',
                   stroke_width=3))
    
    # Return SVG object.
    return SVG(dr.tostring())
    
interact(drawing,
         wall_thickness=(0.0, 0.5, 0.01),
         wall_density  =(0.0, 3.0, 0.01),
         base_width    =(0.0, 1.0, 0.01),
         fluid_height  =(0.0, 1.0, 0.01))

interactive(children=(FloatSlider(value=0.1, description='wall_thickness', max=0.5, step=0.01), FloatSlider(va…

<function __main__.drawing(wall_thickness=0.1, wall_density=1.0, base_width=0.5, fluid_height=0.6)>

The threshold angle $\alpha$ at which the container would fall over because
of gravity is the inverse tangent of the ratio of two lengths,

1. $\frac{L}{2}$, half the length of the base, and
2. $h$, the height of the center of mass.

\begin{equation}
   \alpha = \arctan \frac{L}{2h}
\end{equation}

But $h$ has its maximum possible value, half the height $H$ of the
container, both when the container is completely empty and when it is
completely full.
\begin{equation}
   h_{\text{max}} = \frac{H}{2}
\end{equation}
The threshold tipping angle is minimized in these conditions.
\begin{equation}
   \alpha_{\text{min}} = \arctan \frac{L}{4H}
\end{equation}
So, somewhere in between completely emtpy and completely full, there must
be a liquid level that gives the container the maximum safe tipping angle,
$\alpha_{\text{max}}$.  This corresponds to the minimum height
$h_{\text{min}}$ of the center of mass.

But how might I compute and expression for $h_{\text{min}}$?

<!--- vim: set tw=75: -->

## Configuration of Notebook

The following javascript-magic cell enables the editing of any cell in the
external editor, gvim.

<!--- vim: set filetype=markdown tw=75 -->



In [5]:
%%javascript

// This cell defines two command-mode shortcuts.
//
// Pressing 'g' in command mode copies the content of the current cell into
// a file and then launches gvim to edit that file.
//
// Pressing 'u' in command mode updates the current cell with the contents
// of the file.
//
// The idea is first to press 'g', then to edit the file with gvim, then to
// exit from gvim, and finally to press 'u' to update the cell from the
// file.

// The 'g' shortcut.
IPython.keyboard_manager.command_shortcuts.add_shortcut('g', {
    handler : function (event) {
        var input = IPython.notebook.get_selected_cell().get_text();
        var cmd = "f = open('.toto.txt', 'w');f.close()";
        if (input != "") {
            cmd = '%%writefile .toto.txt\n' + input;
        }
        IPython.notebook.kernel.execute(cmd);
        cmd = "import os;"
        // Establish a clean PATH that has only operating-system-native
        // executables.  On my machine, having '/opt/miniconda3/bin' in the
        // PATH, as it is by default, causes youcompleteme to malfunction
        // because it finds '/opt/miniconda3/bin/python' before
        // '/usr/bin/python'.
        cmd = cmd + "os.putenv('PATH','/bin:/usr/bin');"
        cmd = cmd + "os.system('gvim .toto.txt')";
        IPython.notebook.kernel.execute(cmd);
        return false;
    }}
);

// The 'u' shortcut.
IPython.keyboard_manager.command_shortcuts.add_shortcut('u', {
    handler : function (event) {
        function handle_output(msg) {
            var ret = msg.content.text;
            IPython.notebook.get_selected_cell().set_text(ret);
        }
        var callback = {'output': handle_output};
        var cmd = "f = open('.toto.txt', 'r');print(f.read())";
        IPython.notebook.kernel.execute(cmd, {iopub: callback},
                                        {silent: false});
        return false;
    }}
);

// vim: set filetype=javascript tw=75 sw=4:



<IPython.core.display.Javascript object>

The following cell brings in the libraries needed for linear regression
and for plotting.

In [6]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

The following cell allows the use of widgets.

In [7]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [8]:
def ch(H, W, D, h):
    return H, W, D, h

In [9]:
interact(ch, H=10, W=5, D=2, h=7);

interactive(children=(IntSlider(value=10, description='H', max=30, min=-10), IntSlider(value=5, description='W…